# Майнор ВШЭ
## Прикладные задачи анализа данных 2020
## Семинар 13: Диалоговые системы

**Диалоговая (вопросно-ответная) система** - это система, которая умеет обрабатывать введенный пользователем вопрос на каком-либо языке и давать осмысленный ответ.

Например так:

<center>
  
![img](https://demotos.ru/sites/default/files/caricatures/2019-03-30-1553943418.jpg)
</center>

В последнее время диалоговые системы набирают популярность.  
Question Answering -- одно из самых активно развивающихся направлений в NLP.  
Многие крупные IT-компании создают диалоговых помощников, таких как Siri (Apple), Cortana (Microsoft), Alexa (Amazon) и Алиса (Яндекс).  

Сейчас доступно огромное количество открытых вопросно-ответных датасетов (субтитры фильмов, диалоги, данные с Википедии и др)

Самый популярный датасет -- SQUAD от Стэнфорда  

<div>
<img src="https://i1.wp.com/syncedreview.com/wp-content/uploads/2019/04/Screen-Shot-2019-03-29-at-4.18.21-PM.png?resize=1058%2C440&ssl=1" width="400"/>
</div>

https://rajpurkar.github.io/SQuAD-explorer/   
На нем тестируют различные архитектуры нейронных сетей (например, BERT). 

In [0]:
!pip install tensorflow==1.15

In [0]:
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json

In [0]:
import json
import numpy as np
from keras.models import Sequential, Model
from keras.layers import Embedding, Dropout, Dense, Activation, CuDNNLSTM
from keras.layers import LSTM, Bidirectional,Input
from keras.layers import concatenate
from keras.models import Sequential
from keras import optimizers
from keras.layers import CuDNNLSTM, Dense, Bidirectional, Conv1D, MaxPooling1D, Dropout, GlobalAveragePooling1D, LSTM
from keras import backend as K
from keras.layers.core import Layer  
from keras import initializers, regularizers, constraints  
from keras import backend as K

from keras import backend as K, initializers, regularizers, constraints
from keras.engine.topology import Layer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import keras
import gensim

import tensorflow as tf

In [77]:
print("TF version:", tf.__version__)
print("GPU is", "available" if tf.test.is_gpu_available() else "NOT AVAILABLE")
print(tf.test.gpu_device_name())

TF version: 1.15.0
GPU is available
/device:GPU:0


In [0]:
train = json.load(open('train-v2.0.json'))
dev = json.load(open('dev-v2.0.json'))

In [32]:
import nltk
nltk.download('punkt')

--2020-04-29 20:16:35--  https://raw.githubusercontent.com/yandexdataschool/Practical_DL/fall18/week11_dssm/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9814 (9.6K) [text/plain]
Saving to: ‘utils.py.1’

utils.py.1          100%[===================>]   9.58K  --.-KB/s    in 0s      

2020-04-29 20:16:35 (91.6 MB/s) - ‘utils.py.1’ saved [9814/9814]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

**Задание 1.**
Выведите тип данных train, dev и примеры хранящихся данных.

In [0]:
#------------------------------------------------
# Ваш код здесь
#------------------------------------------------


Видим, что для каждого элемента в train есть:

*   context
*   questiion
*   text


**Посмотрим на примере:**

![](https://miro.medium.com/max/2000/1*0KDRs1_tYlqIs6_FYiaNBQ.png)

Задача состоит в том, чтобы найти в параграфе ответ на заданный вопрос.

**Задание 2.**
Выведите первый вопрос и первый ответ из train.

In [0]:
#------------------------------------------------
# Ваш код здесь
#------------------------------------------------

**Задание 3.**  
Реализуйте предобработку (удаление стоп-слов, пунктуации и тп)

In [0]:
from string import punctuation
from nltk.corpus import stopwords
punct = punctuation + '«»—…“”*№–'

def normalize(text):
    #------------------------------------------------
    # Ваш код здесь
    #------------------------------------------------ 

    return words

Как организован датасет?  
Даны тексты из Википедии и к какому-то параграфу этого текста задан вопрос и из этого же параграфа извлечен ответ.  
В версии 2.0 добавились также вопросы, на которые нет ответа, что усложняет задачу для модели, но мы пока будем опускать такие вопросы.

Для каждого ответа даны начало и конец (индексы) в соответствующем параграфе. На этих индексах мы и будем обучаться.

Но для начала нужно все предобработать. Чтобы уменьшить время обучения предобучим FastText от Facebook AI на всех параграфах и вопросах

In [0]:
corpus = []

**Задание 4.**  
Примените предобработку к context, question.   
Если ответа нет, пропускайте этот вопрос.

In [0]:
contexts = []
questions = []

starts = []
ends = []

for instance in train['data']:


    for paragraph in instance['paragraphs']:
        context = paragraph['context']
        corpus.append(normalize(context))
        
        for qas in paragraph['qas']:
                #------------------------------------------------
                # Ваш код здесь
                #------------------------------------------------ 
            
            if qas['is_impossible']:
                #------------------------------------------------
                # Ваш код здесь
                #------------------------------------------------ 
            
            for answer in qas['answers']:
                #------------------------------------------------
                # Ваш код здесь
                #------------------------------------------------ 

                corpus.append(normalize(question))

Для отложенной выборки сохраним исходные параграфы и индексы вопросов. Они пригодятся для тестирования.

In [0]:
contexts_dev = []
questions_dev = []

starts_dev = []
ends_dev = []
ids = []
imporssible_ids = []
raw_context = []
for instance in dev['data']:
    for paragraph in instance['paragraphs']:
        context = paragraph['context']
        corpus.append(normalize(context))
        for qas in paragraph['qas']:
            if qas['is_impossible']:
                imporssible_ids.append(qas['id'])
                continue
            
            question = qas['question']
            
            for answer in qas['answers']:
                
                start = answer['answer_start']
                end = len(answer['text'])
                contexts_dev.append(normalize(context))
                questions_dev.append(normalize(question))
                starts_dev.append(start)
                ends_dev.append(end)
                corpus.append(normalize(question))
                ids.append(qas['id'])
                raw_context.append(context)

Обучаем FastText

In [0]:
ft = gensim.models.FastText(corpus, size=200, sg=1)

**Задание 5.**  
Построим словарь id2word и word2id.

In [0]:
vocab = set()

for context in contexts:
    vocab.update(context)

for question in questions:
    vocab.update(question)
    
for context in contexts_dev:
    vocab.update(context)

for question in questions_dev:
    vocab.update(question)

#------------------------------------------------
# Ваш код здесь
#------------------------------------------------

id2word = {}
word2id = {}

Создадим матрицу с эмбеддингами всех слов. Потом подадим её в Embedding слой нейронной сети.

In [12]:
embeddings = np.zeros((len(vocab) + 1, 200))

for i in range(1, len(vocab)+1):
    try:
        embeddings[i] = ft[id2word[i]]
    except KeyError:
        embeddings[i] = np.random.rand((200))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


Теперь преобразуем все слова индексы и приведем все к одной длине (максимальной).

**Контекст**

In [0]:
contexts_le = [[word2id[word] for word in context] for context in contexts]
max_len = max([len(c) for c in contexts])

X_train_context = pad_sequences(contexts_le, max_len, padding='post')

In [0]:
contexts_le_dev = [[word2id.get(word, 0) for word in context] for context in contexts_dev]

X_dev_context = pad_sequences(contexts_le_dev, max_len, padding='post')

**Вопросы**

**Задание 6.**.  
Для вопросов реализуйте самостоятельно.

In [0]:
#------------------------------------------------
# Ваш код здесь
#------------------------------------------------


Зададим параметры для нейронки.

In [0]:
vocab_size = len(vocab) + 1
embedding_vector_length = 200

max_span_begin = np.max(starts)
max_span_end = np.max(ends)


In [0]:
starts = np.array(starts)  
ends = np.array(ends) 

starts_dev = np.array(starts_dev) 
ends_dev = np.array(ends_dev)

Теперь основная часть.  
У нашей нейронной сети будет два входа и два выхода. 


In [21]:
# !pip install tensorflow==1.15

# Первый вход
context_input = Input(shape=(max_len, ), name='context_input')
emb_c = Embedding(input_dim=vocab_size, output_dim=200, weights=[embeddings], 
              input_length=max_len, trainable=False)(context_input)

lstm_out_c = Bidirectional(CuDNNLSTM(50,  return_sequences=True,))(emb_c)
drop_1 = Dropout(0.1)(lstm_out_c)

# Второй вход
ques_input = Input(shape=(max_len_q, ), name='ques_input')
emb_q = Embedding(input_dim=vocab_size, output_dim=200, weights=[embeddings], 
              input_length=max_len_q, trainable=False)(ques_input)
lstm_out_q = Bidirectional(CuDNNLSTM(50,return_sequences=True,) )(emb_q)
drop_2 = Dropout(0.1)(lstm_out_q)

# merger model
merge_layer = concatenate([drop_1, drop_2], axis=1)
biLSTM_s = Bidirectional(CuDNNLSTM(10,))(merge_layer)
biLSTM_e = Bidirectional(CuDNNLSTM(10,))(merge_layer)

# Выход 1
softmax_1 = Dense(max_span_begin+1, activation='softmax', name='start')(biLSTM_s)

# Выход 2
softmax_2 = Dense(max_span_end+1, activation='softmax', name='end')(biLSTM_e)

model = Model(inputs=[context_input, ques_input], outputs=[softmax_1, softmax_2])


model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
context_input (InputLayer)      (None, 650)          0                                            
__________________________________________________________________________________________________
ques_input (InputLayer)         (None, 40)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 650, 200)     22157600    context_input[0][0]              
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 40, 200)      22157600    ques_input[0][0]                 
___________

Попробуем обучить модель

In [22]:
%%time
validation_data=({'context_input': X_dev_context,
                  'ques_input':X_dev_question}, 
                 {'start': starts_dev,
                  'end': ends_dev})


training_data=({'context_input': X_train_context,
                'ques_input':X_train_question}, 
                 {'start': starts,
                  'end': ends})

model.fit(training_data[0], training_data[1], batch_size=1024,  epochs=100, shuffle=True,
          validation_data=(validation_data[0], validation_data[1]))


Train on 86821 samples, validate on 20302 samples
Epoch 1/100
86821/86821 [==============================] - 81s 933us/step - loss: 12.0491 - start_loss: 7.4450 - end_loss: 4.6010 - start_accuracy: 0.0045 - end_accuracy: 0.0776 - val_loss: 10.9411 - val_start_loss: 6.9453 - val_end_loss: 3.9950 - val_start_accuracy: 0.0230 - val_end_accuracy: 0.0735
Epoch 2/100
86821/86821 [==============================] - 75s 863us/step - loss: 10.7193 - start_loss: 6.7978 - end_loss: 3.9211 - start_accuracy: 0.0256 - end_accuracy: 0.0817 - val_loss: 10.7254 - val_start_loss: 6.8215 - val_end_loss: 3.9031 - val_start_accuracy: 0.0230 - val_end_accuracy: 0.0735
Epoch 3/100
86821/86821 [==============================] - 73s 841us/step - loss: 10.6127 - start_loss: 6.7287 - end_loss: 3.8840 - start_accuracy: 0.0256 - end_accuracy: 0.0817 - val_loss: 10.6887 - val_start_loss: 6.8015 - val_end_loss: 3.8864 - val_start_accuracy: 0.0230 - val_end_accuracy: 0.0735
Epoch 4/100
86821/86821 [==================

KeyboardInterrupt: ignored

Точность лучше всего оценить с помощью скрипта evaluation.py на страничке SQuAd. 

https://rajpurkar.github.io/SQuAD-explorer/

**Задание 7.**. 
Предиктим на val data

In [23]:
#------------------------------------------------
# Ваш код здесь
#------------------------------------------------
preds = 

20302/20302 [==============================] - 59s 3ms/step


Предиктим распределения для каждого выхода. Делаем argmax(), чтобы достать самый вероятный класс(индекс).

In [0]:
starts_pred = preds[0].argmax(axis=1)
ends_pred = preds[1].argmax(axis=1)

По индексам достаем текст из нетронутых текстов из отложенной выборки. Для плохих вопросов, просто добавляем пустые строки (что на самом деле не совсем правильно).

In [0]:
pred_dict = {}
for i in range(len(starts_pred)):
    cont = raw_context[i]
    span = cont[starts_pred[i]:starts_pred[i]+ends_pred[i]]
    pred_dict[ids[i]] = span

for idx in imporssible_ids:
    pred_dict[idx] = ""

In [26]:
pred_dict

{'56ddde6b9a695914005b9628': 'The ',
 '56ddde6b9a695914005b9629': 'The ',
 '56ddde6b9a695914005b962a': 'The ',
 '56ddde6b9a695914005b962b': 'The ',
 '56ddde6b9a695914005b962c': 'The ',
 '56dddf4066d3e219004dad5f': 'The ',
 '56dddf4066d3e219004dad60': 'The ',
 '56dddf4066d3e219004dad61': 'The ',
 '56dde0379a695914005b9636': 'The ',
 '56dde0379a695914005b9637': 'The ',
 '56dde0ba66d3e219004dad75': 'In t',
 '56dde0ba66d3e219004dad76': 'In t',
 '56dde0ba66d3e219004dad77': 'In t',
 '56dde1d966d3e219004dad8d': 'Befo',
 '56dde27d9a695914005b9651': 'The ',
 '56dde27d9a695914005b9652': 'The ',
 '56dde2fa66d3e219004dad9b': 'The ',
 '56de0f6a4396321400ee257f': 'Soon',
 '56de0ffd4396321400ee258d': 'One ',
 '56de0ffd4396321400ee258e': 'One ',
 '56de0ffd4396321400ee258f': 'One ',
 '56de10b44396321400ee2593': 'Some',
 '56de10b44396321400ee2594': 'Some',
 '56de10b44396321400ee2595': 'Some',
 '56de11154396321400ee25aa': 'Seve',
 '56de148dcffd8e1900b4b5bc': 'Robe',
 '56de148dcffd8e1900b4b5bd': 'Robe',
 

**Задание 8.**.  
Сохраняем результат в json файл.

In [0]:
#------------------------------------------------
# Ваш код здесь
#------------------------------------------------
import json


Для оценки точности запустим скрипт 

In [0]:
!python3 evaluate-v2.0.py dev-v2.0.json prediction.json

## Полезные ссылки:
- SQuAD dataset: https://rajpurkar.github.io/SQuAD-explorer/

- Official BERT version from Google: https://github.com/google-research/bert

- DeepPavlov, a library that has an Open-Domain QA system:
https://medium.com/deeppavlov/open-domain-question-answering-with-deeppavlov-c665d2ee4d65

- Как устроена Алиса:
https://habr.com/company/yandex/blog/349372/